In [1]:
import pandas as pd
import sqlalchemy as sql
import matplotlib.pyplot
import numpy

### NOTE: You will need to replicate the OONI db

Please see [our instructions for replicating the OONI DB on Amazon EC2](https://gist.github.com/elsehow/7e26f4949cb8fca3bf80b5de5be7f106).

Once you've done that, you can run this code on your EC2 instance

In [7]:
sql_engine = sql.create_engine('postgresql:///metadb')

# Compute interference rate for 2019

First, we find all reports for 2019.

In [16]:
reports_2019 = pd.read_sql_query("SELECT * from report where to_char(report.test_start_time, 'YYYY') = '2019'", sql_engine)

In [21]:
total_reports_2019 = len(reports_2019)
total_reports_2019

3940141

Next, we'll find the total number of reports made in each country. This will allow us to compute our rates

In [20]:
cc_by_num_reports_2019 = reports_2019.groupby("probe_cc").nunique()

In [25]:
cc_by_num_reports_2019 = cc_by_num_reports_2019['report_no']

In [28]:
# SANITY CHECK:
# number of reports across all countries should equal the total number of reports
assert cc_by_num_reports_2019.sum() == total_reports_2019

Now, we'll find the number of *confirmed* and *anomolous* interference events in each country.

In [40]:
# some sample queries to help us see the schema...
pd.read_sql_query("SELECT * from ooexpl_wc_input_counts LIMIT 3", sql_engine)

,input,confirmed_count,anomaly_count,failure_count,total_count,test_day,bucket_date,probe_cc,probe_asn
0,https://www.sendspace.com/,0,0,0,1,2019-08-24,2019-08-24,US,701
1,https://www.sendspace.com/,0,0,0,2,2019-08-24,2019-08-24,US,7018
2,https://www.sendspace.com/,0,0,0,3,2019-08-24,2019-08-24,US,7922


In [32]:
probe_results_2019 = pd.read_sql_query("SELECT * from ooexpl_wc_input_counts where to_char(ooexpl_wc_input_counts.test_day, 'YYYY') = '2019'", sql_engine)

In [39]:
cc_by_probe_results_2019 = probe_results_2019.groupby('probe_cc').nunique()
cc_by_probe_results_2019.head()

,input,confirmed_count,anomaly_count,failure_count,total_count,test_day,bucket_date,probe_cc,probe_asn
probe_cc,,,,,,,,,
AD,38,1,1,1,1,1,1,1,1
AE,1610,3,1,1,3,59,60,1,6
AF,168,1,1,1,1,5,5,1,4
AG,28,1,1,1,1,1,1,1,1
AL,590,1,1,1,2,19,19,1,10


We'll now merge the probe results with the number of reports, so that we can compute the rates of confirmed/anomolous reports for each country.

In [55]:
df = cc_by_probe_results_2019[['confirmed_count', 'anomaly_count']]

In [60]:
interference_measurements_by_country=\
pd.DataFrame({'probe_cc': cc_by_num_reports_2019.index,
             'total': cc_by_num_reports_2019.values})\
    .merge(df, how='outer', left_on='probe_cc', right_on='probe_cc')

In [61]:
interference_measurements_by_country.head()

,probe_cc,total,confirmed_count,anomaly_count
0,AD,75,1.0,1.0
1,AE,6826,3.0,1.0
2,AF,1551,1.0,1.0
3,AG,48,1.0,1.0
4,AI,19,NaN,NaN


In [ ]:
# %load ../helpers.py
import datetime
import json

def dated_filename (fn, ext='.csv'):
    today = datetime.date.today()
    return '{}-{}{}'.format(fn, today, ext)

def load_json (fn):
    with open(fn, 'r') as myfile:
        data=myfile.read()
    return json.loads(data)


In [2]:
interference_measurements_by_country.to_csv(dated_filename('interference-measurements-by-country-2010'))

NameError: name 'interference_measurements_by_country' is not defined